In [45]:
import json
from google.colab import files
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import (Dense, Embedding, Flatten, Input, concatenate)
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler

In [29]:
pip install faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.5 MB/s eta 0:00:00


In [17]:
try:
    from google.colab import drive

    !gdown "1Mgb_IhQHroQDuG6lQ039F6NF8Yeuy6QN"

    nutrition_json = '../content/sample_data/nutrition_data.json'
except ImportError:
    nutrition_json = '../sample_data/nutrition_data.json'

Downloading...
From: https://drive.google.com/uc?id=1Mgb_IhQHroQDuG6lQ039F6NF8Yeuy6QN
To: /content/nutrition_data.json
100% 66.4M/66.4M [00:00<00:00, 98.0MB/s]


In [18]:
df_nutrition = pd.read_json(nutrition_json)
df_nutrition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Age                      200000 non-null  int64 
 1   Weight                   200000 non-null  int64 
 2   Gender                   200000 non-null  object
 3   Height                   200000 non-null  int64 
 4   Activity_Level           200000 non-null  object
 5   Goal                     200000 non-null  object
 6   Estimated_Calories       200000 non-null  int64 
 7   Estimated_Carbohydrates  200000 non-null  int64 
 8   Estimated_Protein_Min    200000 non-null  int64 
 9   Estimated_Protein_Max    200000 non-null  int64 
 10  Estimated_Fat            200000 non-null  int64 
dtypes: int64(8), object(3)
memory usage: 16.8+ MB


In [24]:

for index, row in df_nutrition.iterrows():
    if row['Activity_Level'] in ['Sedentary', 'Light']:
        df_nutrition.at[index, 'Activity_Level'] = 'Beginner'
    if row['Activity_Level'] in ['Moderate', 'Active']:
        df_nutrition.at[index, 'Activity_Level'] = 'Intermediate'
    if row['Activity_Level'] in ['Very Active', 'Extra Active']:
        df_nutrition.at[index, 'Activity_Level'] = 'Expert'

# Print the modified DataFrame
print(df_nutrition)

        Age  Weight Gender  Height Activity_Level              Goal  \
0        58      61      m     181       Beginner   Maintain Weight   
1        58      61      m     181       Beginner  Mild Weight Loss   
2        58      61      m     181       Beginner       Weight Loss   
3        58      61      m     181       Beginner  Mild Weight Gain   
4        58      61      m     181       Beginner       Gain Weight   
...     ...     ...    ...     ...            ...               ...   
199995   41      51      m     171       Beginner   Maintain Weight   
199996   41      51      m     171       Beginner  Mild Weight Loss   
199997   41      51      m     171       Beginner       Weight Loss   
199998   41      51      m     171       Beginner  Mild Weight Gain   
199999   41      51      m     171       Beginner       Gain Weight   

        Estimated_Calories  Estimated_Carbohydrates  Estimated_Protein_Min  \
0                     1941                      291                  

In [25]:
df_nutrition.head()

,Age,Weight,Gender,Height,Activity_Level,Goal,Estimated_Calories,Estimated_Carbohydrates,Estimated_Protein_Min,Estimated_Protein_Max,Estimated_Fat
0,58,61,m,181,Beginner,Maintain Weight,1941,291,72,169,32
1,58,61,m,181,Beginner,Mild Weight Loss,1746,291,65,152,29
2,58,61,m,181,Beginner,Weight Loss,1533,291,57,134,25
3,58,61,m,181,Beginner,Mild Weight Gain,2426,291,90,212,40
4,58,61,m,181,Beginner,Gain Weight,2814,291,105,246,46


In [41]:
from faker import Faker
import tensorflow as tf
import numpy as np

fake = Faker()

def generate_dummy_data(num_samples):
    data = []
    for _ in range(num_samples):
        profile = {
            "Age": fake.random_int(min=18, max=80),
            "Weight": fake.random_int(min=40, max=150),
            "Gender": fake.random_element(["m", "f"]),
            "Height": fake.random_int(min=140, max=200),
            "Activity_Level": fake.random_element(["Sedentary", "Lightly Active", "Moderately Active", "Very Active"]),
            "Goal": fake.random_element(["Mild Weight Loss", "Weight Loss", "Mild Weight Gain"]),
        }
        data.append(profile)
    return data

# Generating dummy data
num_samples = 3
dummy_data = generate_dummy_data(num_samples)

# Creating TensorFlow tensors from the generated data
age = tf.constant([profile["Age"] for profile in dummy_data], dtype=tf.float32)
weight = tf.constant([profile["Weight"] for profile in dummy_data], dtype=tf.float32)
gender = tf.constant([1.0 if profile["Gender"] == "f" else 0.0 for profile in dummy_data], dtype=tf.float32)
height = tf.constant([profile["Height"] for profile in dummy_data], dtype=tf.float32)
activity_level = tf.constant([
    {"Sedentary": 0, "Lightly Active": 1, "Moderately Active": 2, "Very Active": 3}[profile["Activity_Level"]]
    for profile in dummy_data
], dtype=tf.float32)
goal = tf.constant([
    {"Mild Weight Loss": 0, "Weight Loss": 1, "Mild Weight Gain": 2}[profile["Goal"]]
    for profile in dummy_data
], dtype=tf.float32)

# Print the generated data tensors
print("Age:", age)
print("Weight:", weight)
print("Gender:", gender)
print("Height:", height)
print("Activity Level:", activity_level)
print("Goal:", goal)

Age: tf.Tensor([49. 45. 51.], shape=(3,), dtype=float32)
Weight: tf.Tensor([136.  90.  90.], shape=(3,), dtype=float32)
Gender: tf.Tensor([1. 1. 1.], shape=(3,), dtype=float32)
Height: tf.Tensor([147. 184. 172.], shape=(3,), dtype=float32)
Activity Level: tf.Tensor([3. 1. 2.], shape=(3,), dtype=float32)
Goal: tf.Tensor([0. 2. 2.], shape=(3,), dtype=float32)


In [42]:
FEATURES = ['Age', 'Weight', 'Gender', 'Height', 'activity_level', 'goal']
LABEL_ENCODER = dict()

In [43]:
def get_col_to_encode(*dataframes):
    cols = set()

    for dataframe in dataframes:
        dataframe_cols = dataframe.select_dtypes(exclude=[np.number])
        cols.update(dataframe_cols)

        for col in dataframe_cols.columns:
            if col != 'name':
                LABEL_ENCODER[col] = LABEL_ENCODER.get(col, LabelEncoder().fit(dataframe[col]))

    if 'name' in cols:
        cols.remove('name')

    return cols

**EDA**

In [44]:
df_nutrition.head()

,Age,Weight,Gender,Height,Activity_Level,Goal,Estimated_Calories,Estimated_Carbohydrates,Estimated_Protein_Min,Estimated_Protein_Max,Estimated_Fat
0,58,61,m,181,Beginner,Maintain Weight,1941,291,72,169,32
1,58,61,m,181,Beginner,Mild Weight Loss,1746,291,65,152,29
2,58,61,m,181,Beginner,Weight Loss,1533,291,57,134,25
3,58,61,m,181,Beginner,Mild Weight Gain,2426,291,90,212,40
4,58,61,m,181,Beginner,Gain Weight,2814,291,105,246,46


In [46]:
df_nutrition.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Age                      200000 non-null  int64 
 1   Weight                   200000 non-null  int64 
 2   Gender                   200000 non-null  object
 3   Height                   200000 non-null  int64 
 4   Activity_Level           200000 non-null  object
 5   Goal                     200000 non-null  object
 6   Estimated_Calories       200000 non-null  int64 
 7   Estimated_Carbohydrates  200000 non-null  int64 
 8   Estimated_Protein_Min    200000 non-null  int64 
 9   Estimated_Protein_Max    200000 non-null  int64 
 10  Estimated_Fat            200000 non-null  int64 
dtypes: int64(8), object(3)
memory usage: 16.8+ MB


In [48]:
print(
    df_nutrition.Age.value_counts(),
    df_nutrition.Weight.value_counts(),
    df_nutrition.Gender.value_counts(),
    df_nutrition.Height.value_counts(),
    df_nutrition.Activity_Level.value_counts(),
    sep='\n\n'
)

29    4990
52    4945
47    4910
31    4885
37    4870
18    4855
48    4845
25    4830
19    4795
40    4785
20    4730
51    4725
27    4725
43    4720
53    4720
41    4705
49    4705
28    4695
34    4685
23    4680
60    4670
33    4665
38    4650
35    4640
24    4630
50    4625
21    4615
56    4605
58    4585
45    4570
54    4545
30    4540
57    4540
32    4525
44    4510
46    4505
55    4500
39    4495
26    4475
36    4465
42    4315
22    4280
59    4250
Name: Age, dtype: int64

59     3875
80     3845
82     3810
77     3795
94     3750
51     3740
69     3735
74     3735
93     3730
67     3725
64     3700
88     3685
45     3665
97     3660
89     3650
54     3635
48     3625
100    3620
76     3615
92     3605
86     3590
66     3590
49     3580
91     3580
52     3580
46     3575
61     3565
79     3560
78     3555
56     3550
75     3540
71     3525
90     3525
98     3525
65     3525
84     3520
68     3510
55     3505
85     3505
70     3500
83     3495
60     349

**Encoding**

In [51]:
def encode_hist_nut(df_nutrition):
    encoded_df_nutrition = df_nutrition.copy()

    columns_to_encode = get_col_to_encode(encoded_df_nutrition) # Inplace encode

    for col in columns_to_encode:

        if col in encoded_df_nutrition.columns:
            encoded_df_nutrition[col] = LABEL_ENCODER[col].transform(encoded_df_nutrition[col])


    return encoded_df_nutrition

df_nutrition_copy = encoded_df_nutrition(df_nutrition)

NameError: ignored